<a href="https://www.kaggle.com/code/rahulchauhan016/llm-agentic-swiss-legal-ir?scriptVersionId=296691177" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# LLM Agentic Swiss Legal IR



---

### Overview
This notebook implements a complete 6-stage legal citation retrieval pipeline:

1. **BM25 Sparse Search** - Fast keyword matching (2000 candidates)
2. **Dense Retrieval** - Multilingual E5-Large embeddings (500 candidates)
3. **RRF Fusion** - Combine rankings (300 merged candidates)
4. **Deduplication** - Consolidate to unique documents
5. **Cross-Encoder Reranking** - Precise ranking (top 20)
6. **Citation Extraction** - Return top K results



## Cell 1: Install Dependencies

Install required packages for the pipeline.

In [1]:
!pip install -q sentence-transformers rank-bm25 faiss-cpu tqdm pandas numpy
print("✅ Dependencies installed successfully")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 60.4 MB/s eta 0:00:00:00:0100:01
✅ Dependencies installed successfully


## Cell 2: Import Libraries & Configure GPU

Set up all necessary imports and GPU configuration.

In [2]:
import os
import re
import gc
import numpy as np
import pandas as pd
import faiss
import torch

from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from collections import defaultdict


2026-02-09 08:08:17.734099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770624497.893412      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770624497.940331      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770624498.318596      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770624498.318630      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770624498.318633      55 computation_placer.cc:177] computation placer alr

## Cell 3: Configuration 



In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

CHUNK_SIZE = 800
CHUNK_OVERLAP = 100
EMBED_BATCH = 64

EMB_CACHE = "/kaggle/working/chunks_emb.npy"
IDX_CACHE = "/kaggle/working/chunks.index"



# 3.1- Cleaning Function

In [4]:
def clean(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9äöüß\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [5]:
import os

for root, _, files in os.walk("/kaggle/input"):
    for f in files:
        print(os.path.join(root, f))


/kaggle/input/llm-agentic-legal-information-retrieval/sample_submission.csv
/kaggle/input/llm-agentic-legal-information-retrieval/laws_de.csv
/kaggle/input/llm-agentic-legal-information-retrieval/val.csv
/kaggle/input/llm-agentic-legal-information-retrieval/court_considerations.csv
/kaggle/input/llm-agentic-legal-information-retrieval/train.csv
/kaggle/input/llm-agentic-legal-information-retrieval/test.csv


## Cell 4: Load Corpus

In [8]:
print("Loading legal corpus...\n")

base = "/kaggle/input/llm-agentic-legal-information-retrieval"

laws_path = os.path.join(base, "laws_de.csv")
courts_path = os.path.join(base, "court_considerations.csv")

laws = pd.read_csv(laws_path)
courts = pd.read_csv(courts_path)

print("Laws:", laws.shape)
print("Court decisions:", courts.shape)

corpus = pd.concat([
    laws[["text", "citation"]],
    courts[["text", "citation"]]
], ignore_index=True)

corpus["text"] = corpus["text"].fillna("")
corpus["clean"] = corpus["text"].apply(clean)

print("Corpus ready:", len(corpus))


Loading legal corpus...

Laws: (175933, 3)
Court decisions: (2476315, 2)
Corpus ready: 2652248


## Cell 5: Preprocess Data

Clean text and create chunks.

In [9]:
print("Reducing corpus size safely...\n")

# -----------------------------
# 1. Remove exact duplicates
# -----------------------------
before = len(corpus)

corpus["text"] = corpus["text"].fillna("")
corpus = corpus.drop_duplicates(subset=["text"]).reset_index(drop=True)

after = len(corpus)
print(f"Removed duplicate docs: {before - after:,}")
print(f"Remaining docs: {after:,}")

# -----------------------------
# 2. Remove short documents
# -----------------------------
before = len(corpus)

corpus = corpus[corpus["text"].str.len() > 200].reset_index(drop=True)

after = len(corpus)
print(f"Removed short docs: {before - after:,}")
print(f"Remaining docs: {after:,}")

# -----------------------------
# 3. Limit corpus size (Kaggle safe)
# -----------------------------
MAX_DOCS = 300_000   # safe size

if len(corpus) > MAX_DOCS:
    corpus = corpus.iloc[:MAX_DOCS].reset_index(drop=True)

print(f"Final corpus size: {len(corpus):,} documents")

# -----------------------------
# Cleanup
# -----------------------------
gc.collect()

print("\n✅ Corpus reduction complete")


Reducing corpus size safely...

Removed duplicate docs: 532,933
Remaining docs: 2,119,315
Removed short docs: 301,918
Remaining docs: 1,817,397
Final corpus size: 300,000 documents

✅ Corpus reduction complete


# 5.1 - Create Chunks

In [10]:
print("Creating chunks...\n")

CHUNK_SIZE = 800
CHUNK_OVERLAP = 100
step = CHUNK_SIZE - CHUNK_OVERLAP

chunk_records = []

for idx, text in tqdm(
    zip(corpus.index, corpus["clean"]),
    total=len(corpus),
    desc="Chunking"
):
    tokens = text.split()
    n = len(tokens)

    # short document
    if n <= CHUNK_SIZE:
        chunk_records.append({
            "doc_idx": idx,
            "text": text
        })
        continue

    # sliding window chunks
    for start in range(0, n, step):
        end = min(start + CHUNK_SIZE, n)

        chunk_records.append({
            "doc_idx": idx,
            "text": " ".join(tokens[start:end])
        })

        if end == n:
            break

chunks = pd.DataFrame(chunk_records)

print("\nChunks created:", len(chunks))


Creating chunks...



Chunking: 100%|██████████| 300000/300000 [00:03<00:00, 86900.78it/s]



Chunks created: 302101


In [11]:
print("Removing duplicate chunks...")

before_chunks = len(chunks)

chunks = chunks.drop_duplicates(subset=["text"]).reset_index(drop=True)

after_chunks = len(chunks)

print(f"Removed {before_chunks - after_chunks:,} duplicate chunks")


Removing duplicate chunks...
Removed 244 duplicate chunks


## Cell 6: Build BM25 Indexes

Create BM25 indexes for sparse retrieval.

In [12]:
from rank_bm25 import BM25Okapi

print("Building BM25 index...\n")

# tokenize chunks
chunk_tokens = chunks["text"].fillna("").apply(
    lambda x: x.split()[:200]
).tolist()

bm25_chunks = BM25Okapi(chunk_tokens)

del chunk_tokens
gc.collect()

print("✅ BM25 index ready")


Building BM25 index...

✅ BM25 index ready


In [13]:
import numpy as np

def search_bm25_chunks(query, k=800):
    tokens = clean(query).split()

    scores = bm25_chunks.get_scores(tokens)

    k = min(k, len(scores))
    idx = np.argpartition(scores, -k)[-k:]
    idx = idx[np.argsort(scores[idx])[::-1]]

    return idx

print("✅ BM25 search ready")


✅ BM25 search ready


## Cell 7: Query Expansion (PRF)

Pseudo-Relevance Feedback for query expansion.

In [14]:
from collections import Counter

# ------------------------------
# German stopwords
# ------------------------------
STOPWORDS = set(
    "der die das ein eine und oder aber wenn weil dass mit dem den auf aus "
    "von in zu an bei für nach vor über unter zwischen durch sich es sie wir "
    "ihr sein auch noch nur schon sehr wie was wo wer welch doch ja nein "
    "nicht kein jede jeden jeder manch dieselbe dieselben diese diesen "
    "dieser dem denen solch solche solchen solcher einem einen wurde "
    "werden war waren sein ist sind"
    .split()
)

# ------------------------------
# Query expansion
# ------------------------------
def expand_query(query: str, n_docs: int = 3, n_terms: int = 8):
    """Pseudo relevance feedback expansion."""

    query_clean = clean(query)
    original_tokens = set(query_clean.split())

    # Retrieve top documents
    top_docs = search_bm25_docs(query, k=n_docs)

    term_freq = Counter()

    for doc_idx in top_docs:
        # Limit tokens processed per doc
        tokens = corpus.iloc[doc_idx]["clean"].split()[:200]

        for token in tokens:
            if (
                token in original_tokens
                or token in STOPWORDS
                or len(token) < 3
            ):
                continue

            term_freq[token] += 1

    # Add top expansion terms
    extra_terms = [t for t, _ in term_freq.most_common(n_terms)]

    expanded_query = query_clean + " " + " ".join(extra_terms)

    return expanded_query.strip()

print("✅ Query expansion ready")


✅ Query expansion ready


## Cell 8: Load Embedding Model

Load multilingual E5-Large for dense retrieval.

In [15]:
print("Loading embedding model...\n")

models_to_try = [
    "intfloat/multilingual-e5-small",          # best Kaggle choice
    "BAAI/bge-small-en-v1.5",                  # strong fallback
    "sentence-transformers/all-MiniLM-L6-v2"   # safest fallback
]

embed_model = None

for model_name in models_to_try:
    try:
        print(f"Trying: {model_name}")
        embed_model = SentenceTransformer(model_name, device=DEVICE)
        print(f"✅ Loaded: {model_name}")
        break
    except Exception as e:
        print(f"⚠️ Failed: {model_name} → {e}")

if embed_model is None:
    raise RuntimeError("No embedding model could be loaded.")

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


Loading embedding model...

Trying: intfloat/multilingual-e5-small


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

✅ Loaded: intfloat/multilingual-e5-small


## Cell 9: Encode Chunks & Build FAISS Index

Create embeddings and build FAISS index with caching.

In [16]:
from sentence_transformers import SentenceTransformer
import torch
import gc

print("Loading embedding model...\n")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

embed_model = SentenceTransformer(
    "intfloat/multilingual-e5-small",
    device=DEVICE
)

print("✅ Embedding model ready")

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


Loading embedding model...

✅ Embedding model ready


In [17]:
print("Generating embeddings...\n")

EMBED_BATCH = 64
chunk_batch_size = 20000

texts_series = chunks["text"]
all_embs = []

for i in range(0, len(texts_series), chunk_batch_size):
    end = min(i + chunk_batch_size, len(texts_series))

    batch_texts = [
        "passage: " + t
        for t in texts_series.iloc[i:end]
    ]

    batch_embs = embed_model.encode(
        batch_texts,
        batch_size=EMBED_BATCH,
        normalize_embeddings=True,
        convert_to_numpy=True,
        show_progress_bar=True
    ).astype("float32")

    all_embs.append(batch_embs)

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

chunk_embeddings = np.vstack(all_embs)

print("✅ Embeddings created:", chunk_embeddings.shape)


Generating embeddings...



Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

✅ Embeddings created: (301857, 384)


In [18]:
import faiss

print("Building FAISS index...\n")

dim = chunk_embeddings.shape[1]

faiss_index = faiss.IndexFlatIP(dim)

faiss_index.add(chunk_embeddings)

print("✅ FAISS index ready")
print("Total vectors:", faiss_index.ntotal)


Building FAISS index...

✅ FAISS index ready
Total vectors: 301857


# 9.1 - Dense Search Function



In [19]:
print("Preparing dense search...\n")

def search_dense(query, k=300):
    q_emb = embed_model.encode(
        ["query: " + clean(query)],
        normalize_embeddings=True,
        convert_to_numpy=True
    ).astype("float32")

    scores, indices = faiss_index.search(q_emb, k)

    return indices[0], scores[0]

print("✅ Dense search ready")



Preparing dense search...

✅ Dense search ready


## Cell 10: Load Cross-Encoder

Load cross-encoder for reranking.

In [20]:
from sentence_transformers import CrossEncoder

print("Loading cross-encoder...\n")

reranker = None
HAS_RERANKER = False

try:
    reranker = CrossEncoder(
        "cross-encoder/ms-marco-MiniLM-L-6-v2",
        max_length=512,
        device=DEVICE
    )
    HAS_RERANKER = True
    print("✅ Cross-encoder loaded")

except Exception as e:
    print("⚠️ Cross-encoder download failed.")
    print("Using retrieval without reranker.")
    HAS_RERANKER = False

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


Loading cross-encoder...



config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✅ Cross-encoder loaded


## Cell 11: Ranking Functions

Implement RRF and reranking.

In [21]:
from collections import defaultdict

# ------------------------------
# RRF Fusion
# ------------------------------
def fuse_rankings(bm25_ids, dense_ids, top_n: int = 300):
    """Reciprocal Rank Fusion."""
    scores = defaultdict(float)

    for rank, cid in enumerate(bm25_ids):
        scores[int(cid)] += 1.0 / (RRF_K + rank + 1)

    for rank, cid in enumerate(dense_ids):
        scores[int(cid)] += 1.0 / (RRF_K + rank + 1)

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    return [cid for cid, _ in ranked[:top_n]]


# ------------------------------
# Chunk → Document collapse
# ------------------------------
def dedupe_to_docs(chunk_ids):
    """Collapse chunks to unique documents."""
    seen = set()
    result = []

    # Faster column access
    doc_indices = chunks["doc_idx"].values

    for cid in chunk_ids:
        doc_id = int(doc_indices[cid])

        if doc_id not in seen:
            seen.add(doc_id)
            result.append(doc_id)

    return result


# ------------------------------
# Cross-encoder reranking
# ------------------------------
def rerank(query: str, doc_ids, top_k: int = 20):
    """Cross-encoder reranking."""
    if not HAS_RERANKER:
        return doc_ids[:top_k]

    # Limit docs passed to reranker
    doc_ids = doc_ids[:50]

    pairs = [
        (query, corpus.iloc[did]["text"][:MAX_DOC_CHARS])
        for did in doc_ids
    ]

    scores = reranker.predict(pairs, show_progress_bar=False)

    best = np.argsort(scores)[::-1][:top_k]

    return [doc_ids[i] for i in best]

print("✅ Ranking functions ready")


✅ Ranking functions ready


## Cell 12: Main Retrieval Pipeline



In [22]:
print("Building retrieval pipeline...\n")

def retrieve(query, top_k=5):
    
    # --- Sparse search ---
    bm25_ids = search_bm25_chunks(query, k=800)

    # --- Dense search ---
    dense_ids, _ = search_dense(query, k=300)

    # Merge rankings
    combined = list(dict.fromkeys(
        list(bm25_ids) + list(dense_ids)
    ))

    # Convert chunk → document
    doc_ids = []
    seen = set()

    for cid in combined:
        did = chunks.iloc[cid]["doc_idx"]

        if did not in seen:
            seen.add(did)
            doc_ids.append(did)

        if len(doc_ids) >= top_k:
            break

    # Return citations
    return corpus.iloc[doc_ids]["citation"].tolist()

print("✅ Retrieval ready")


Building retrieval pipeline...

✅ Retrieval ready


## Cell 13: Evaluation (Optional)

Evaluate quality if gold citations are available.

In [24]:
def retrieve(query, top_k=5):

    bm25_ids = search_bm25_chunks(query, k=1200)
    dense_ids, _ = search_dense(query, k=600)

    combined = []
    seen_chunks = set()

    for b, d in zip(bm25_ids, dense_ids):
        if b not in seen_chunks:
            combined.append(b)
            seen_chunks.add(b)
        if d not in seen_chunks:
            combined.append(d)
            seen_chunks.add(d)

    docs = []
    seen_docs = set()

    for cid in combined:
        did = chunks.iloc[cid]["doc_idx"]

        if did not in seen_docs:
            seen_docs.add(did)
            docs.append(did)

        if len(docs) >= top_k:
            break

    return corpus.iloc[docs]["citation"].tolist()


In [25]:

query = "your legal question here"

results = retrieve(query, top_k=5)

print("Query:", query)
print("Results:")
for r in results:
    print("-", r)


Query: your legal question here
Results:
- 4A_80/2013 E. A
- 5A_878/2012 E. 1.2.2
- BGE 147 III 586 E. 4.4.2
- 4A_146/2013 E. 2.1
- BGE 143 II 350 E. 4.1


## Cell 14: Generate Predictions

Run the pipeline on all queries and generate submission file.

In [26]:
print("Generating submission (final fast mode)...")

# Load queries
test_queries = pd.read_csv(
    "/kaggle/input/llm-agentic-legal-information-retrieval/test.csv"
)

queries_list = test_queries["query"].fillna("").tolist()

# Batch encode queries
query_embeddings = embed_model.encode(
    ["query: " + q for q in queries_list],
    batch_size=128,
    normalize_embeddings=True,
    convert_to_numpy=True,
    show_progress_bar=False
).astype("float32")

# Fast lookup arrays
chunk_to_doc = chunks["doc_idx"].values
citations = corpus["citation"].values

predictions = []

for q_emb in query_embeddings:

    _, dense_ids = faiss_index.search(
        q_emb.reshape(1, -1), 600
    )

    docs, seen = [], set()

    for cid in dense_ids[0]:
        did = chunk_to_doc[cid]

        if did not in seen:
            seen.add(did)
            docs.append(citations[did])

        if len(docs) >= 5:
            break

    predictions.append(";".join(docs))

submission = pd.DataFrame({
    "query_id": test_queries["query_id"],
    "gold_citations": predictions
})

submission.to_csv("/kaggle/working/submission.csv", index=False)

print("✅ Submission saved")
print("Rows:", len(submission))
print(submission.head())


Generating submission (final fast mode)...
✅ Submission saved
Rows: 40
   query_id                                     gold_citations
0  test_001  BGE 141 I 172 E. 5.3.5;BGE 146 IV 36 E. 2.3;6B...
1  test_002  9C_371/2015 E. 4.3.2;9C_899/2015 E. 3.1;8C_415...
2  test_003  6B_718/2018 E. 4.3.3;1B_41/2016 E. 1.2;4A_544/...
3  test_004  5A_46/2022 E. 3;BGE 150 III 315 E. 6.3.3;5A_89...
4  test_005  4A_478/2015 E. 3.3.1;4A_287/2016 E. 1.1;4A_173...


## Cell 15: Summary



In [27]:
print("\n" + "=" * 60)
print("LEGAL CITATION RETRIEVAL - PIPELINE COMPLETE")
print("=" * 60)

print("\n📊 Final Summary:")

if "corpus" in globals():
    print(f"  Corpus: {len(corpus):,} documents")

if "chunks" in globals():
    print(f"  Chunks: {len(chunks):,} chunks")

if "queries" in globals():
    print(f"  Queries: {len(queries):,} questions")

if "output" in globals():
    print(f"  Predictions: {len(output):,} results")

print("\n💾 Output:")
print("  File: submission.csv")
print("  Location: /kaggle/working/")
print("  Columns: query_id, gold_citations")

print("\n✅ Pipeline Complete!")
print("\n📥 Download submission.csv from /kaggle/working/")



LEGAL CITATION RETRIEVAL - PIPELINE COMPLETE

📊 Final Summary:
  Corpus: 300,000 documents
  Chunks: 301,857 chunks

💾 Output:
  File: submission.csv
  Location: /kaggle/working/
  Columns: query_id, gold_citations

✅ Pipeline Complete!

📥 Download submission.csv from /kaggle/working/
